In [3]:
import cv2
import mediapipe as mp
import numpy as np
import time
import math
 
class AlcoholDetectionSystem:
    def __init__(self):
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(max_num_faces=1)
        self.mp_drawing = mp.solutions.drawing_utils
        self.last_mar_time = time.time()
        self.mar_threshold = 0.5
        self.gaze_deviation_threshold = 5
        self.pupil_diameter_threshold = 6  # mm
        self.head_yaw_threshold = 15
        self.head_pitch_threshold = 30
        self.alcohol_alert_threshold = 3  # Number of criteria to trigger alert
   
    def calculate_eye_aspect_ratio(self, landmarks):
        left_eye = np.array([[landmarks.landmark[p].x, landmarks.landmark[p].y]
                             for p in [33, 160, 158, 133, 153, 144]])
        right_eye = np.array([[landmarks.landmark[p].x, landmarks.landmark[p].y]
                              for p in [362, 385, 387, 263, 373, 380]])
        left_ear = self.eye_aspect_ratio(left_eye)
        right_ear = self.eye_aspect_ratio(right_eye)
        return (left_ear + right_ear) / 2
   
    def eye_aspect_ratio(self, eye_points):
        A = np.linalg.norm(eye_points[1] - eye_points[5])
        B = np.linalg.norm(eye_points[2] - eye_points[4])
        C = np.linalg.norm(eye_points[0] - eye_points[3])
        ear = (A + B) / (2.0 * C)
        return ear
   
    def calculate_mouth_aspect_ratio(self, landmarks):
        mouth = np.array([[landmarks.landmark[p].x, landmarks.landmark[p].y]
                          for p in [61, 185, 40, 11]])
        mar = self.mouth_aspect_ratio(mouth)
        return mar
   
    def mouth_aspect_ratio(self, mouth_points):
        A = np.linalg.norm(mouth_points[0] - mouth_points[2])
        B = np.linalg.norm(mouth_points[1] - mouth_points[3])
        mar = (A + B) / 2.0
        return mar
   
    def get_head_pose(self, landmarks):
        nose = np.array([landmarks.landmark[1].x, landmarks.landmark[1].y])
        chin = np.array([landmarks.landmark[152].x, landmarks.landmark[152].y])
        left_eye = np.array([landmarks.landmark[33].x, landmarks.landmark[33].y])
        right_eye = np.array([landmarks.landmark[362].x, landmarks.landmark[362].y])
 
        yaw = math.degrees(math.atan2(right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]))
        pitch = math.degrees(math.atan2(chin[1] - nose[1], chin[0] - nose[0]))
       
        return yaw, pitch
   
    def alcohol_detection_logic(self, gaze_deviation, pupil_diameter, mar, head_yaw, head_pitch):
        criteria_met = 0
       
        if abs(gaze_deviation) > self.gaze_deviation_threshold:
            criteria_met += 1
        if pupil_diameter > self.pupil_diameter_threshold:
            criteria_met += 1
        if mar > self.mar_threshold:
            criteria_met += 1
        if abs(head_yaw) > self.head_yaw_threshold and abs(head_pitch) > self.head_pitch_threshold:
            criteria_met += 1
       
        if criteria_met >= self.alcohol_alert_threshold:
            return True  # Alcohol detected
        return False
   
    def process_frame(self, frame):
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(rgb_frame)
       
        if results.multi_face_landmarks:
            for landmarks in results.multi_face_landmarks:
                self.mp_drawing.draw_landmarks(frame, landmarks, self.mp_face_mesh.FACEMESH_TESSELATION)
               
                mar = self.calculate_mouth_aspect_ratio(landmarks)
                gaze_deviation = self.get_head_pose(landmarks)[0]
                pupil_diameter = self.calculate_eye_aspect_ratio(landmarks)
                head_yaw, head_pitch = self.get_head_pose(landmarks)
               
                alcohol_detected = self.alcohol_detection_logic(gaze_deviation, pupil_diameter, mar, head_yaw, head_pitch)
               
                # Display information on screen
                cv2.putText(frame, f"MAR: {mar:.2f}", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                cv2.putText(frame, f"Gaze Deviation: {gaze_deviation:.2f}", (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                cv2.putText(frame, f"Pupil Diameter: {pupil_diameter:.2f} mm", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                cv2.putText(frame, f"Head Yaw: {head_yaw:.2f}", (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                cv2.putText(frame, f"Head Pitch: {head_pitch:.2f}", (30, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
               
                if alcohol_detected:
                    cv2.putText(frame, "ALCOHOL DETECTED!", (250, 250), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3, cv2.LINE_AA)
       
        return frame
   
    def run_detection(self):
        cap = cv2.VideoCapture(0)
       
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
           
            frame = self.process_frame(frame)
            cv2.imshow("Alcohol Detection System", frame)
           
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
       
        cap.release()
        cv2.destroyAllWindows()
 
if __name__ == "__main__":
    detector = AlcoholDetectionSystem()
    detector.run_detection()
 
 